In [128]:
from tkinter import ttk
import tkinter as tk
import pandas as pd
import numpy as np
import json
import csv
import os
import time
import operator
import random
import pygame
from keras.models import load_model

In [133]:
##################
# Global variables
##################
total_num_songs = 0
songs_found = 0
percent_songs_found = 0.0
most_popular_genre = "None"
#pygame.init()

##################
# Load in data
##################
genres_combined = {}
with open('../data/genres_combined.json') as f:
    genres_combined = json.load(f)

all_songs = []
for key,value in genres_combined.items():
    for song in genres_combined[key]:
        if song['song'] not in all_songs:
            all_songs.append(song['song'])
            total_num_songs += 1
#print(total_num_songs)
del all_songs

# Dropdown menu options
options1 = []
options2 = []
options3 = []
options4 = []

X = pd.read_csv('../data/X_123.csv')
y = pd.read_csv('../data/y_4.csv')

for col in X.columns:
    if col[:2] == '1_':
        options1.append(col[2:])
    if col[:2] == '2_':
        options2.append(col[2:])
    if col[:2] == '3_':
        options3.append(col[2:])

for col in y.columns:
    options4.append(col[2:])

chord_list_for_weighted_probabilities = []
counts_for_weighted_probabilities = []
for key,value in genres_combined.items():
    a,b,c,d = key.split(',')
    if a in options1 and b in options2 and c in options3 and d in options4:
        key3 = ','.join([x for x in key.split(',')[:3]])
        if key3 in chord_list_for_weighted_probabilities:
            index1 = chord_list_for_weighted_probabilities.index(key3)
            counts_for_weighted_probabilities[index1] += len(genres_combined[key])
        else:
            chord_list_for_weighted_probabilities.append(key3)
            counts_for_weighted_probabilities.append(len(genres_combined[key]))

##################
# Load in model
##################
model = load_model('../models/model_full_X.h5')

In [134]:
##################
# Define chords
##################
"""
print(np.unique(options1+options2))
['1' '142' '16' '164' '17' '2' '242' '26' '264' '265' '27' '3' '36' '364'
 '365' '37' '4' '442' '46' '464' '47' '5' '5/2' '5/3' '5/5' '5/6' '542'
 '543' '56' '56/5' '56/6' '564' '564/6' '565' '565/6' '57' '57/2' '57/4'
 '57/5' '57/6' '6' '642' '66' '664' '665' '67' '7' '7/3' '7/5' '7/6' '77']
"""
chorddict = {}
chorddict['1'] =     ['C', 'E', 'G']
chorddict['142'] =   ['B0', 'C', 'E', 'G']
chorddict['16'] =    ['E', 'G', 'C1']
chorddict['164'] =   ['G', 'C1', 'E1']
chorddict['17'] =    ['C', 'E', 'G', 'B']
chorddict['2'] =     ['D', 'F', 'A']
chorddict['242'] =   ['C', 'D', 'F', 'A']
chorddict['26'] =    ['F', 'A', 'D1']
chorddict['264'] =   ['A0', 'D', 'F']
chorddict['265'] =   ['F', 'A', 'C1', 'D1']
chorddict['27'] =    ['D', 'F', 'A', 'C1']
chorddict['3'] =     ['E', 'G', 'B']
chorddict['36'] =    ['G', 'B', 'E1']
chorddict['364'] =   ['B0', 'E', 'G']
chorddict['365'] =   ['G', 'B', 'D1', 'E1']
chorddict['37'] =    ['E', 'G', 'B', 'D1']
chorddict['4'] =     ['F', 'A', 'C1']
chorddict['442'] =   ['E', 'F', 'A', 'C1']
chorddict['46'] =    ['A0', 'C', 'F']
chorddict['464'] =   ['C', 'F', 'A']
chorddict['47'] =    ['F', 'A', 'C1', 'E1']
chorddict['5'] =     ['G', 'B', 'D1']
chorddict['5/2'] =   ['A0', 'C_s', 'E']
chorddict['5/3'] =   ['B0', 'D_s', 'F_s']
chorddict['5/5'] =   ['D', 'F_s', 'A']
chorddict['5/6'] =   ['E', 'G_s', 'B']
chorddict['542'] =   ['F', 'G', 'B', 'D1']
chorddict['543'] =   ['D', 'F', 'G', 'B']
chorddict['56'] =    ['B0', 'D', 'G']
chorddict['56/5'] =  ['F_s', 'A', 'C1', 'D1']
chorddict['56/6'] =  ['G_s', 'B', 'D1', 'E1']
chorddict['564'] =   ['D', 'G', 'B']
chorddict['564/6'] = ['B0', 'E', 'G_s']
chorddict['565'] =   ['B0', 'D', 'F', 'G']
chorddict['565/6'] = ['G_s', 'B', 'D1', 'E1']
chorddict['57'] =    ['G', 'B', 'D1', 'F1']
chorddict['57/2'] =  ['A0', 'C_s', 'E', 'G']
chorddict['57/4'] =  ['C', 'E', 'G', 'Bb']
chorddict['57/5'] =  ['D', 'F_s', 'A', 'C1']
chorddict['57/6'] =  ['E', 'G_s', 'B', 'D1']
chorddict['6'] =     ['A0', 'C', 'E']
chorddict['642'] =   ['G', 'A', 'C1', 'E1']
chorddict['66'] =    ['C', 'E', 'A']
chorddict['664'] =   ['E', 'A', 'C1']
chorddict['665'] =   ['C', 'E', 'G', 'A']
chorddict['67'] =    ['A0', 'C', 'E', 'G']
chorddict['7'] =     ['B0', 'D', 'F']
chorddict['7/3'] =   ['D_s', 'F_s', 'A']
chorddict['7/5'] =   ['F_s', 'A', 'C1']
chorddict['7/6'] =   ['G_s', 'B', 'D1']
chorddict['7/7'] =   ['Bb0', 'C_s', 'E']

# pygame needs numbers instead of chord names, with middle C as 60
str2intdict = {}
str2intdict['A0'] =   57
str2intdict['Bb0'] =  58
str2intdict['B0'] =   59
str2intdict['C'] =    60
str2intdict['C_s'] =  61
str2intdict['D'] =    62
str2intdict['D_s'] =  63
str2intdict['E'] =    64
str2intdict['F'] =    65
str2intdict['F_s'] =  66
str2intdict['G'] =    67
str2intdict['G_s'] =  68
str2intdict['A'] =    69
str2intdict['Bb'] =   70
str2intdict['B'] =    71
str2intdict['C1'] =   72
str2intdict['C_s1'] = 73
str2intdict['D1'] =   74
str2intdict['D_s1'] = 75
str2intdict['E1'] =   76
str2intdict['F1'] =   77

chorddict2 = {}
for key,value in chorddict.items():
    chorddict2[key] = [str2intdict[x] for x in value]

In [147]:
##################
# Create GUI
##################
root = tk.Tk()
root.geometry("800x700") # size

# Add instructions to top
instructions1 = tk.Label(root,
            text="\nWelcome to Jesse's Song Composer v1.0!")
instructions1.grid(row=0, column=0, columnspan=12, pady=5) 
instructions2 = tk.Label(root,
            text=("\nInstructions:" + \
                 "\n\t1. Select chords from the dropdowns. Alternatively, you can click the 'Randomize' button to select completely random " + \
                        "\n\t    chord combinations or select the 'Let model choose' button to let the pre-trained model predict a good progression."
                 "\n\t2. When you have selected the 4 chords you like, click the 'Make my song' button. Metrics about the progression and a " + \
                        "\n\t    list of songs containing that progression will populate in the table at the bottom." + \
                 "\n\t3. Press 'Play my song' to hear the chords played." + \
                 "\n\n"),
            justify='left')
instructions2.grid(row=1, column=0, columnspan=12, pady=5)

##################
# Functions
##################
def populate_table():
    chord1 = clicked1.get()
    chord2 = clicked2.get()
    chord3 = clicked3.get()
    chord4 = clicked4.get()
    # clear table
    tree.delete(*tree.get_children())
    # make dict key
    chordstr = chord1+','+ chord2+','+ chord3+','+ chord4
    # check dict
    rows = genres_combined.get(chordstr, None)
    # populate table
    if rows:
        gdict = {}
        percent_songs_found = round(100*len(rows)/total_num_songs, 2)
        label_table1.config(text=str(percent_songs_found)+"% of songs have this chord progression")
        for row in rows:
            for g in row['genre']:
                if g not in gdict.keys():
                    if g != 'Unknown':
                        gdict[g] = 1
                else:
                    gdict[g] += 1
            tree.insert("", tk.END, values=(row['song'], row['artist'], row['url'], ', '.join(row['genre'])))
        if len(gdict.items()):
            label_table2.config(text="'" + \
                                max(gdict.items(), key=operator.itemgetter(1))[0] + \
                                "' is the most popular genre with this progression")
        else:
            label_table2.config(text="'Unknown' is the most popular genre with this progression")
    else:
        label_table1.config(text="0.0% of songs have this chord progression")
        label_table2.config(text="'None' is the most popular genre with this progression")
    
def randomize():
    clicked1.set(random.sample(options1, 1)[0])
    show1()
    clicked2.set(random.sample(options2, 1)[0])
    show2()
    clicked3.set(random.sample(options3, 1)[0])
    show3()
    clicked4.set(random.sample(options4, 1)[0])
    show4()

def fill_chords_with_model():
    # pick random index from X for first 3 chords
    #i = random.randint(0, len(X)-1)
    #c1c, c2c, c3c = X.columns[(X.iloc[i]==1)]
    
    # rather than go with a fully random set of first 3 chords above, let's use
    #     weighted probabilities based on what first 3 chords belong to the most songs
    first3 = random.choices(population=chord_list_for_weighted_probabilities,
                            weights=counts_for_weighted_probabilities,
                            k=1)[0]

    c1c, c2c, c3c = first3.split(',')
    iarr = np.zeros((len(X.columns),))
    c1i = list(X.columns).index('1_'+c1c)
    c2i = list(X.columns).index('2_'+c2c)
    c3i = list(X.columns).index('3_'+c3c)
    iarr[c1i] = 1
    iarr[c2i] = 1
    iarr[c3i] = 1
    # convert pandas'Int64Index' to list of ints, then take the first
    i = X[(X==iarr).all(axis=1)].index.tolist()[0]

    clicked1.set(c1c) #c1c[2:]
    show1()
    clicked2.set(c2c) #c2c[2:]
    show2()
    clicked3.set(c3c) #c3c[2:]
    show3()
    #c4s = y.columns[(y.iloc[i]>0)]
    #c4s = c4s.tolist()
    #c4c = random.sample(c4s, 1)[0]
    # Instead of the above for c4, let's use the model we trained to predict a 4th chord
    threechords = np.array(X.iloc[i,:]).reshape((1,143))
    ypred = model.predict(threechords)[0]
    # filter out garbage
#     ypred[ypred < 0.1] = 0
#     fourth = random.choices(
#         population=y.columns,
#         weights=ypred,
#         k=1)[0]
    #top3index = ypred.argsort()[-3:][::-1]
    maxindex = np.argmax(ypred)
    fourth = y.columns[maxindex]
    #print(ypred, maxindex, ypred[maxindex], fourth)
    clicked4.set(fourth[2:])
    show4()

def compose_song():
    # TODO: create and display the chords
    #create_score()
    #display_score()
    populate_table()

def play_song():
    pygame.midi.quit()
    pygame.midi.init()
    player = pygame.midi.Output(pygame.midi.get_default_output_id())
    player.set_instrument(0)

#     notes = chorddict[clicked1.get()]
#     sound1 = pygame.mixer.Sound(os.path.join(os.getcwd(), '..', 'piano', notes[0]+'.wav'))
#     sound1.play()
#     time.sleep(0.7)
#     sound1.stop()
#     sound2 = pygame.mixer.Sound(os.path.join(os.getcwd(), '..', 'piano', notes[1]+'.wav'))
#     sound2.play()
#     time.sleep(0.7)
#     sound2.stop()
#     sound3 = pygame.mixer.Sound(os.path.join(os.getcwd(), '..', 'piano', notes[2]+'.wav'))
#     sound3.play()
#     time.sleep(0.7)
#     sound3.stop()
#     if len(notes) == 4:
#         sound4 = pygame.mixer.Sound(os.path.join(os.getcwd(), '..', 'piano', notes[3]+'.wav'))
#         sound4.play()
#         time.sleep(0.7)
#         sound4.stop()

    # better method than the above! don't need .wav files and can play chords at the same time
    # 1st chord
    notes = chorddict2[clicked1.get()]
    for note in notes:
        player.note_on(note, 127)
    time.sleep(1)
    for note in notes:
        player.note_off(note, 127)
    # 2nd chord
    notes = chorddict2[clicked2.get()]
    for note in notes:
        player.note_on(note, 127)
    time.sleep(1)
    for note in notes:
        player.note_off(note, 127)
    # 3rd chord
    notes = chorddict2[clicked3.get()]
    for note in notes:
        player.note_on(note, 127)
    time.sleep(1)
    for note in notes:
        player.note_off(note, 127)
    # 4th chord
    notes = chorddict2[clicked4.get()]
    for note in notes:
        player.note_on(note, 127)
    time.sleep(1)
    for note in notes:
        player.note_off(note, 127)

    del player
    pygame.midi.quit()

# Change the label text
def show1(value=''):
    label1.config(text=clicked1.get())
def show2(value=''):
    label2.config(text=clicked2.get())
def show3(value=''):
    label3.config(text=clicked3.get())
def show4(value=''):
    label4.config(text=clicked4.get())

# Initialize dropdown text
clicked1 = tk.StringVar()
clicked2 = tk.StringVar()
clicked3 = tk.StringVar()
clicked4 = tk.StringVar()

# Set initial menu text set to most popular chord progression
clicked1.set("1")
clicked2.set("5")
clicked3.set("6")
clicked4.set("4")

##################
# Chord Dropdowns, Buttons, and Labels
##################
# Create dropdown menu 1
drop1 = tk.OptionMenu(root, clicked1, *options1, command=show1)
drop1.grid(row=3, column=0, columnspan=3, pady=10)
# Create button 1
#button1 = tk.Button(root, text="Select Chord #1", command=show1)
#button1.grid(row=4, column=0, columnspan=3, pady=10)
# Create Label 1
label1 = tk.Label(root, text=" ")
label1.grid(row=5, column=0, columnspan=3, pady=10)

# Create dropdown menu 2
drop2 = tk.OptionMenu(root, clicked2, *options2, command=show2)
drop2.grid(row=3, column=3, columnspan=3, pady=10)
# Create button 2
#button2 = tk.Button(root, text="Select Chord #2", command=show2)
#button2.grid(row=4, column=3, columnspan=3, pady=10)
# Create Label 2
label2 = tk.Label(root, text=" ")
label2.grid(row=5, column=3, columnspan=3, pady=10)

# Create dropdown menu 3
drop3 = tk.OptionMenu(root, clicked3, *options3, command=show3)
drop3.grid(row=3, column=6, columnspan=3, pady=10)
# Create button 3
#button3 = tk.Button(root, text="Select Chord #3", command=show3)
#button3.grid(row=4, column=6, columnspan=3, pady=10)
# Create Label 3
label3 = tk.Label(root, text=" ")
label3.grid(row=5, column=6, columnspan=3, pady=10)

# Create dropdown menu 4
drop4 = tk.OptionMenu(root, clicked4, *options4, command=show4)
drop4.grid(row=3, column=9, columnspan=3, pady=10)
# Create button 4
#button4 = tk.Button(root, text="Select Chord #4", command=show4)
#button4.grid(row=4, column=9, columnspan=3, pady=10)
# Create Label 4
label4 = tk.Label(root, text=" ")
label4.grid(row=5, column=9, columnspan=3, pady=10)

show1()
show2()
show3()
show4()

##################
# Buttons
##################
# Random button
button_random = tk.Button(text="Randomize", command=randomize)
button_random.grid(row=6, column=3, columnspan=3, pady=10)
# Model button
button_model = tk.Button(text="Let model choose", command=fill_chords_with_model)
button_model.grid(row=6, column=6, columnspan=3, pady=10)

# Make song button
button_make = tk.Button(text="Make my song", font=('arial', 10, 'bold'), command=compose_song)
button_make.grid(row=7, column=3, columnspan=3, pady=10)
# Play song button
button_play = tk.Button(text="Play my song", command=play_song)
button_play.grid(row=7, column=6, columnspan=3, pady=10)

##################
# Create table
##################
empty_space1 = tk.Label(root, text="")
empty_space1.grid(row=9, column=0, columnspan=12, pady=10)
# Table label
label_table1 = tk.Label(root, text=str(percent_songs_found)+"% of songs have this chord progression")
label_table1.grid(row=10, column=0, columnspan=12, pady=3)
label_table2 = tk.Label(root, text="'" + str(most_popular_genre)+"' is the most popular genre with this progression")
label_table2.grid(row=12, column=0, columnspan=12, pady=3)
label_table3 = tk.Label(root, text="Songs with a similar progression")
label_table3.grid(row=14, column=0, columnspan=12, pady=3)
# Table
tree = ttk.Treeview(root, column=("c1", "c2", "c3", "c4"), show='headings')
tree.column("#1", anchor=tk.CENTER)
tree.heading("#1", text="Song")
tree.column("#2", anchor=tk.CENTER)
tree.heading("#2", text="Artist")
tree.column("#3", anchor=tk.CENTER)
tree.heading("#3", text="URL")
tree.column("#4", anchor=tk.CENTER)
tree.heading("#4", text="Genres")
tree.grid(row=15, column=0, columnspan=12, pady=5)

root.title("Jesse's Song Composer")
root.mainloop()

In [113]:
pygame.midi.quit()